This notebook provides a tutorial on how to use the library.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# Datasets

Datasets management is made simple. You can view the available datasets:

In [3]:
from gsitk.datasets.datasets import DatasetManager

dm = DatasetManager()
dm.view_datasets()

- pl04:
    	 Downloaded: True
    	 # instances: 4000

- semeval14:
    	 Downloaded: True
    	 # instances: 7838

- semeval13:
    	 Downloaded: True
    	 # instances: 6259

- imdb:
    	 Downloaded: True
    	 # instances: 50000

- vader:
    	 Downloaded: True
    	 # instances: 4200

- sst:
    	 Downloaded: True
    	 # instances: 11855

- sentiment140:
    	 Downloaded: True
    	 # instances: 1600000

- imdb_unsup:
    	 Downloaded: True
    	 # instances: 50000

- multidomain:
    	 Downloaded: True
    	 # instances: 38548




Preparing the data is done once. For all datasets:

In [ ]:
data = dm.prepare_datasets()

In [10]:
data.keys()

dict_keys(['imdb', 'vader', 'semeval13', 'sst', 'multidomain', 'imdb_unsup', 'pl04', 'semeval14', 'sentiment140'])

And for only one dataset:

In [4]:
data = dm.prepare_datasets(['imdb', 'semeval13', 'semeval14'])

Data is a dict, in which each value is a pandas DataFrame with the dataset.

In [12]:
data['imdb'].head()

,id,fold,text,polarity,rating
0,10239,train,"[i, disagree, with, anyone, who, donet, like, ...",1,10
1,10337,train,"[this, movie, was, extremely, funny, ,, i, wou...",1,9
2,3881,train,"[this, movie, grabbed, me, with, the, incredib...",1,9
3,3548,train,"[just, saw, this, movie, version, of, frank, l...",1,8
4,9453,train,"[deliverance, is, the, fascinating, ,, hauntin...",1,8


# Preprocessing

GSITK has utilities for preprocessing:

In [38]:
from gsitk.preprocess import simple, pprocess_twitter

text = "My grandmother is an apple. Please, believe me!"
twitter_text = "@POTUS please let me enter to the USA #thanks"

print(simple.clean_str(text))
print(pprocess_twitter.tokenize(twitter_text))

my grandmother is an apple . please , believe me !
<user> please let me enter to the usa <allcaps> <hastag> thanks


# Features

GSITK has a variety of feature extrators. For exaple, in order to use a word2vec model as feature extractor, write:

In [39]:
from gsitk.features.word2vec import Word2VecFeatures

w2v_feat = Word2VecFeatures(w2v_model_path='/data/w2vmodel_500d_5mc')

Extracting features is made by the method `transform`. All feature extractors implement `transform`.

In [45]:
transformed = w2v_feat.transform(data['imdb']['text'].values)
transformed.shape

(50000, 500)

In [62]:
transformed[0].shape

(500,)

If extracting the features is time consuming, you can save the features locally:

In [47]:
from gsitk.features import features

features.save_features(transformed, 'w2v__sentiment40')

And you can load them later:

In [49]:
features.load_features('w2v__sentiment')

array([[ 0.04839503, -0.03920275,  0.01310699, ..., -0.01793178,
         0.01850573,  0.01894511],
       [ 0.02001294, -0.01502401, -0.0211135 , ..., -0.01764425,
        -0.00566167,  0.02577729],
       [ 0.01879481, -0.04025034, -0.02238391, ..., -0.01603499,
         0.00581812,  0.03437515],
       ..., 
       [ 0.01735126, -0.02752644, -0.02615537, ..., -0.00227182,
         0.00647882,  0.01969421],
       [ 0.01858013, -0.01519343, -0.01451839, ..., -0.00798909,
         0.00773863,  0.04368705],
       [ 0.03160627, -0.0360069 , -0.006861  , ..., -0.01662612,
         0.00133611,  0.0172867 ]])

# Evaluation

The evaluation process uses pipes. Pipe are a way of organizing the different elements of the evaluation. Pipes are represented by EvalTuples, that are a way of specifiying which datasets, features and classifiers we want to evaluate.

For evaluating a set of models that predict from a set of features, `EvalTuple` are specified. The next example evaluates a simple logistic regressions model that uses word2vec features to predict the sentiment of the `IMDB` dataset.

In [72]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier()

# transformed is the features extracted from the IMDB dataset
# to properly evaluate, separate in train and test 
# using the original dataset fold
train_indices = (data['imdb']['fold'] == 'train').values
test_indices =(data['imdb']['fold'] == 'test').values

transformed_train = transformed[train_indices]
transformed_test = transformed[test_indices]


sgd.fit(transformed_train, data['imdb']['polarity'][train_indices])

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

Prepare the model, features and `EvalTuple` for the evaluation.

In [74]:
from gsitk.pipe import Model, Features, EvalTuple

models = [Model(name='sgd', classifier=sgd)]

feats = [Features(name='w2v__imdb_test', dataset='imdb', values=transformed_test)]

ets = [EvalTuple(classifier='sgd', features='w2v__imdb_test', labels='imdb')]

Perform the evaluation!

In [122]:
from gsitk.evaluation.evaluation import Evaluation

ev = Evaluation(datasets=data, features=feats, models=models, tuples=ets)

In [123]:
# run the evaluation
ev.evaluate()

# view the results
ev.results

,Dataset,Features,Model,CV,accuracy,precision_macro,recall_macro,f1_weighted,f1_micro,f1_macro
0,imdb,w2v__imdb_test,sgd,False,0.76164,0.782904,0.76164,0.757075,0.76164,0.757075


# New use case

In [5]:
data_ready = {}
for data_k, data_v in data.items():
    data_ready[data_k] = data_v.copy()
    data_ready[data_k]['text'] = data_v['text'].apply(' '.join).values

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline.fit(data['semeval13']['text'].apply(' '.join).values,
             data['semeval13']['polarity'].values.astype(int))
pipeline.name = 'pipeline_trained'
pipeline.named_steps['vect'].name = 'myvect'
pipeline.named_steps['tfidf'].name = 'mytfidf'
pipeline.named_steps['clf'].name = 'mylogisticregressor'


pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

pipeline2.fit(data['semeval14']['text'].apply(' '.join).values,
             data['semeval14']['polarity'].values.astype(int))
pipeline2.name = 'pipeline_trained2'

In [8]:
from gsitk.evaluation.evaluation import Evaluation

datasets_evaluation = {
    'semeval13': data_ready['semeval13'],
    'semeval14': data_ready['semeval14']
}

ev = Evaluation(tuples=None,
                datasets=datasets_evaluation,
                pipelines=[pipeline, pipeline2])
ev.evaluate()
ev.results

,Dataset,Features,Model,CV,accuracy,precision_macro,recall_macro,f1_weighted,f1_micro,f1_macro,Description
0,semeval13,None,pipeline_trained__semeval13,False,0.93708,0.951259,0.915951,0.936753,0.93708,0.931609,vect(myvect) --> tfidf(mytfidf) --> clf(mylogi...
1,semeval13,None,pipeline_trained2__semeval13,False,0.664105,0.630446,0.583108,0.655742,0.664105,0.596319,vect --> tfidf --> clf
2,semeval14,None,pipeline_trained2__semeval14,False,0.907514,0.91692,0.889563,0.9071,0.907514,0.901148,vect --> tfidf --> clf
3,semeval14,None,pipeline_trained__semeval14,False,0.642429,0.634108,0.580425,0.632416,0.642429,0.593557,vect(myvect) --> tfidf(mytfidf) --> clf(mylogi...
